# TEST OF CATBOOST

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import lightgbm as lgb
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import PolynomialFeatures

from pathlib import Path
from catboost import CatBoostRegressor

In [2]:
data = pd.read_parquet('./data/train.parquet')
data

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.000000
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.693147
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,0.000000
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,1.609438
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,Y2H15027244,48.846028,2.375429,2.302585
...,...,...,...,...,...,...,...,...,...,...,...
928450,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,51.0,2021-08-08 18:00:00,2020-11-29,Y2H20114504,48.839770,2.301980,3.951244
928453,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,1.0,2021-08-09 02:00:00,2020-11-29,Y2H20114504,48.839770,2.301980,0.693147
928456,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,61.0,2021-08-09 08:00:00,2020-11-29,Y2H20114504,48.839770,2.301980,4.127134
928459,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,44.0,2021-08-09 10:00:00,2020-11-29,Y2H20114504,48.839770,2.301980,3.806662


In [3]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X.loc[:, 'year'] = X['date'].dt.year
    X.loc[:, 'month'] = X['date'].dt.month
    X.loc[:, 'day'] = X['date'].dt.day
    X.loc[:, 'weekday'] = X['date'].dt.weekday
    X.loc[:, 'hour'] = X['date'].dt.hour

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"]) 

In [4]:
import problem

X_train, y_train = problem.get_train_data()
X_test, y_test = problem.get_test_data()

## Example of mask

## Using external data

In [5]:
df_ext = pd.read_csv(Path('submissions') / 'starting_kit' / 'external_data copy.csv')
df_ext.head()

,Unnamed: 0,numer_sta,date,pmer,tend,cod_tend,dd,ff,t,td,...,ctype2,hnuage2,nnuage3,ctype3,hnuage3,nnuage4,ctype4,hnuage4,confi,holiday
0,0,7149,2021-01-01 00:00:00,100810,80,1,270,1.8,272.75,272.15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
1,1,7149,2021-01-01 03:00:00,100920,110,3,300,1.7,271.25,270.95,...,3.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1
2,2,7149,2021-01-01 06:00:00,100950,30,3,290,2.6,271.95,271.65,...,6.0,2000.0,6.0,3.0,3000.0,NaN,NaN,NaN,0,1
3,3,7149,2021-01-01 09:00:00,101100,150,2,280,1.7,272.45,272.05,...,3.0,2800.0,NaN,NaN,NaN,NaN,NaN,NaN,0,1
4,4,7149,2021-01-01 12:00:00,101110,30,0,50,1.0,276.95,274.15,...,6.0,570.0,7.0,6.0,810.0,NaN,NaN,NaN,0,1


You can find the detailed documentation for each feature [in this PDF](https://donneespubliques.meteofrance.fr/client/document/doc_parametres_synop_168.pdf) (in French). Here the only feature we consider is the temperature. We will use [pandas.merge_asof](https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.merge_asof.html) to merge on date, using the closest available date (since the sampling between the external data and the counter data).

In [6]:
# In this notebook we define the __file__ variable to be in the same conditions as when running the
# RAMP submission

__file__ = Path('submissions') /  'starting_kit' /  'estimator.py'


def _merge_external_data(X):
    file_path = Path(__file__).parent / 'external_data copy.csv'
    df_ext = pd.read_csv(file_path, parse_dates=['date'])
    X = X.copy()
    # When using merge_asof left frame need to be sorted
    X['orig_index'] = np.arange(X.shape[0])
    X = pd.merge_asof(X.sort_values('date'), df_ext[['date', 't', 'u', 'rr24', 'rr3', 'tend24', 'holiday']].sort_values('date'), on='date')
    # Sort back to the original order
    X = X.sort_values('orig_index')
    del X['orig_index']
    return X
    

In [7]:
X_train_merge = _merge_external_data(X_train)
columns_merge = X_train_merge.columns
X_train_merge

,counter_id,counter_name,site_id,site_name,date,counter_installation_date,counter_technical_id,latitude,longitude,t,u,rr24,rr3,tend24,holiday
0,100049407-353255860,152 boulevard du Montparnasse E-O,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,2018-12-07,Y2H19070373,48.840801,2.333233,285.75,81,0.0,0.0,120.0,0
30,100049407-353255859,152 boulevard du Montparnasse O-E,100049407,152 boulevard du Montparnasse,2020-09-01 01:00:00,2018-12-07,Y2H19070373,48.840801,2.333233,285.75,81,0.0,0.0,120.0,0
31,100036719-104036719,18 quai de l'Hôtel de Ville NO-SE,100036719,18 quai de l'Hôtel de Ville,2020-09-01 01:00:00,2017-07-12,Y2H19027732,48.853720,2.357020,285.75,81,0.0,0.0,120.0,0
32,100036719-103036719,18 quai de l'Hôtel de Ville SE-NO,100036719,18 quai de l'Hôtel de Ville,2020-09-01 01:00:00,2017-07-12,Y2H19027732,48.853720,2.357020,285.75,81,0.0,0.0,120.0,0
33,100063175-353277233,20 Avenue de Clichy NO-SE,100063175,20 Avenue de Clichy,2020-09-01 01:00:00,2020-07-22,Y2H20073268,48.885290,2.326660,285.75,81,0.0,0.0,120.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455129,100057329-103057329,Totem 85 quai d'Austerlitz SE-NO,100057329,Totem 85 quai d'Austerlitz,2021-08-09 23:00:00,2020-02-18,YTH19111508,48.842010,2.367290,291.45,72,-0.1,0.0,160.0,1
455130,100057380-104057380,Totem Cours la Reine E-O,100057380,Totem Cours la Reine,2021-08-09 23:00:00,2020-02-11,YTH19111509,48.864620,2.314440,291.45,72,-0.1,0.0,160.0,1
455119,100057380-103057380,Totem Cours la Reine O-E,100057380,Totem Cours la Reine,2021-08-09 23:00:00,2020-02-11,YTH19111509,48.864620,2.314440,291.45,72,-0.1,0.0,160.0,1
455136,100042374-110042374,Voie Georges Pompidou NE-SO,100042374,Voie Georges Pompidou,2021-08-09 23:00:00,2017-12-15,Y2H21025335,48.848400,2.275860,291.45,72,-0.1,0.0,160.0,1


In [8]:
def confinement(X):
    date = pd.to_datetime(X['date'])
    X.loc[:, ['date_only']] = date
    new_date = [dt.date() for dt in X['date_only']]
    X.loc[:, ['date_only']] = new_date
    mask = ((X['date_only'] >= pd.to_datetime('2020/10/30'))
        & (X['date_only'] <= pd.to_datetime('2020/12/15'))
        | (X['date_only'] >= pd.to_datetime('2021/04/03'))
        & (X['date_only'] <= pd.to_datetime('2021/05/03')))
    X['confi'] = np.where(mask, 1, 0)
    return X

def curfew(X):
    date = pd.to_datetime(X['date'])
    X.loc[:, ['date_only']] = date
    new_date = [dt.date() for dt in X['date_only']]
    X.loc[:, ['date_only']] = new_date
    X.loc[:, ['hour_only']] = date
    new_hour = [dt.hour for dt in X['hour_only']]
    X.loc[:, ['hour_only']] = new_hour
    mask = (
        #First curfew
        (X['date_only'] >= pd.to_datetime('2020/12/15'))
        & (X['date_only'] < pd.to_datetime('2021/01/16'))
        & ((X['hour_only'] >= 20) | (X['hour_only'] <= 6))

        | 
        
        # Second curfew
        (X['date_only'] >= pd.to_datetime('2021/01/16'))
        & (X['date_only'] < pd.to_datetime('2021/03/20'))
        & ((X['hour_only'] >= 18) | (X['hour_only'] <= 6))

        |

        # Third curfew
        (X['date_only'] >= pd.to_datetime('2021/03/20'))
        & (X['date_only'] < pd.to_datetime('2021/05/19'))
        & ((X['hour_only'] >= 19) | (X['hour_only'] <= 6))

        |

        # Fourth curfew
        (X['date_only'] >= pd.to_datetime('2021/05/19'))
        & (X['date_only'] < pd.to_datetime('2021/06/9'))
        & ((X['hour_only'] >= 21) | (X['hour_only'] <= 6))

        |

        # Fifth curfew
        (X['date_only'] >= pd.to_datetime('2021/06/9'))
        & (X['date_only'] < pd.to_datetime('2021/06/20'))
        & ((X['hour_only'] >= 21) | (X['hour_only'] <= 6))
        )
    X['curfew'] = np.where(mask, 1, 0)

    return X.drop(columns=['hour_only', 'date_only'])

def counters_done(X):
    mask1 = (data['counter_name']=='152 boulevard du Montparnasse E-O') & (data['date'] >= pd.to_datetime('2021/01/26')) & (data['date'] <= pd.to_datetime('2021/02/24'))
    mask1bis = (data['counter_name']=='152 boulevard du Montparnasse O-E') & (data['date'] >= pd.to_datetime('2021/01/26')) & (data['date'] <= pd.to_datetime('2021/02/24'))
    mask2 = (data['counter_name']=='20 Avenue de Clichy SE-NO') & (data['date'] >= pd.to_datetime('2021/05/06')) & (data['date'] <= pd.to_datetime('2021/07/21'))
    mask2bis = (data['counter_name']=='20 Avenue de Clichy NO-SE') & (data['date'] >= pd.to_datetime('2021/05/06')) & (data['date'] <= pd.to_datetime('2021/07/21'))
    X.drop(X[mask1].index, inplace=True)
    X.drop(X[mask1bis].index, inplace=True)
    X.drop(X[mask2].index, inplace=True)
    X.drop(X[mask2bis].index, inplace=True)
    return X


Similarly we can wrap this function into a FunctionTranformer to use it in a scikit-learn pipeline. Now let's see whether this additional feature improves the model,

In [17]:
date_encoder = FunctionTransformer(_encode_dates)

confinement_encoder = FunctionTransformer(confinement)
curfew_encoder = FunctionTransformer(curfew)
counters_done_encoder = FunctionTransformer(counters_done)
categorical_encoder = OneHotEncoder(handle_unknown="ignore")
poly_encoder = PolynomialFeatures(degree=2)

categorical_cols = ["counter_name"]
period_cols = ['confi', 'curfew']
date_cols = ['day', 'year', 'weekday', 'month']
hour_col = ['hour']
numeric_cols = ['t', 'u']

preprocessor = ColumnTransformer(
    [
        ("date", StandardScaler(), date_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ('numeric', StandardScaler(), numeric_cols),
        ('period', 'passthrough', period_cols),
        ('hour', PolynomialFeatures(degree=2), hour_col)
    ]
)
regressor = CatBoostRegressor()

pipe =  make_pipeline(
    FunctionTransformer(_merge_external_data, validate=False),
    confinement_encoder,
    curfew_encoder,
    date_encoder,
    preprocessor,
    regressor
)

In [13]:
pipe

Pipeline(steps=[('functiontransformer-1',
                 FunctionTransformer(func=<function _merge_external_data at 0x7faf249479d0>)),
                ('functiontransformer-2',
                 FunctionTransformer(func=<function confinement at 0x7faf24947670>)),
                ('functiontransformer-3',
                 FunctionTransformer(func=<function curfew at 0x7faf24947d30>)),
                ('functiontransformer-4',
                 FunctionTransformer(func=<function _...f70>)),
                ('columntransformer',
                 ColumnTransformer(transformers=[('date', StandardScaler(),
                                                  ['day', 'year', 'weekday',
                                                   'month']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['counter_name']),
                                     

# Parameters tuning : LGBRegressor

Important parameters : https://www.kaggle.com/bextuychiev/lgbm-optuna-hyperparameter-tuning-w-understanding

Control over the tree structure :
 - num_leaves
 - max_depth
 - min_data_in_leaf

 Improve the accuracy :
 - learning_rate and n_estimators, best combination
 - max_bin (default = 255), can improve but risk of overfitting

 Control overfitting :
 - lambda_1 and lambda_2
 - min_gain_to_split

In [10]:
# Get the params
pipe.get_params()

{'memory': None,
 'steps': [('functiontransformer-1',
   FunctionTransformer(func=<function _merge_external_data at 0x7f9d19222670>)),
  ('functiontransformer-2',
   FunctionTransformer(func=<function confinement at 0x7f9d192224c0>)),
  ('functiontransformer-3',
   FunctionTransformer(func=<function curfew at 0x7f9d19222940>)),
  ('functiontransformer-4',
   FunctionTransformer(func=<function _encode_dates at 0x7f9d1916fc10>)),
  ('columntransformer',
   ColumnTransformer(transformers=[('date', StandardScaler(),
                                    ['day', 'year', 'weekday', 'month']),
                                   ('cat', OneHotEncoder(handle_unknown='ignore'),
                                    ['counter_name']),
                                   ('numeric', StandardScaler(), ['t', 'u']),
                                   ('period', 'passthrough', ['confi', 'curfew']),
                                   ('hour', PolynomialFeatures(), ['hour'])])),
  ('catboostregressor', <catb

In [24]:
#Import libraries
import numpy as np
from scipy.stats import randint
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats

n_iter = 60
# Creating the hyperparameter grid
params = {
    'catboostregressor__learning_rate': [0, 0.05, 0.1, 0.15, 0.5],
    'catboostregressor__l2_leaf_reg': [1, 3, 5, 7, 9],
    'catboostregressor__max_depth': [4, 6, 10],
    'catboostregressor__n_estimators': [100, 200, 300],
    'catboostregressor__loss_function' : ['RMSE'],
    'catboostregressor__one_hot_max_size': stats.bernoulli(p=0.2, loc=2),
}

               
#Instantiate RandomSearchCV object
rscv = RandomizedSearchCV(pipe , params, cv =5, n_iter= n_iter, verbose=False)

#Fit the model
rscv.fit(X_train,y_train)

# Print the tuned parameters and score
print(rscv.best_params_)
print(rscv.best_score_)

/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partiti

0:	learn: 1.6169582	total: 235ms	remaining: 1m 10s
1:	learn: 1.5664797	total: 335ms	remaining: 49.9s
2:	learn: 1.5185893	total: 650ms	remaining: 1m 4s
3:	learn: 1.4736524	total: 813ms	remaining: 1m
4:	learn: 1.4318389	total: 914ms	remaining: 53.9s
5:	learn: 1.3911419	total: 988ms	remaining: 48.4s
6:	learn: 1.3542916	total: 1.05s	remaining: 44.1s
7:	learn: 1.3199244	total: 1.11s	remaining: 40.4s
8:	learn: 1.2877579	total: 1.18s	remaining: 38.2s
9:	learn: 1.2567616	total: 1.24s	remaining: 35.9s
10:	learn: 1.2281153	total: 1.3s	remaining: 34.2s
11:	learn: 1.2015247	total: 1.45s	remaining: 34.8s
12:	learn: 1.1767436	total: 1.52s	remaining: 33.5s
13:	learn: 1.1529761	total: 1.57s	remaining: 32.1s
14:	learn: 1.1315919	total: 1.64s	remaining: 31.2s
15:	learn: 1.1113364	total: 1.73s	remaining: 30.8s
16:	learn: 1.0899669	total: 2.28s	remaining: 38s
17:	learn: 1.0704929	total: 2.48s	remaining: 38.9s
18:	learn: 1.0532892	total: 2.58s	remaining: 38.1s
19:	learn: 1.0367424	total: 2.63s	remaining: 3

/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to mat

0:	learn: 1.6223002	total: 94.6ms	remaining: 28.3s
1:	learn: 1.5717720	total: 172ms	remaining: 25.6s
2:	learn: 1.5243160	total: 261ms	remaining: 25.8s
3:	learn: 1.4800936	total: 319ms	remaining: 23.6s
4:	learn: 1.4389799	total: 395ms	remaining: 23.3s
5:	learn: 1.4002626	total: 450ms	remaining: 22.1s
6:	learn: 1.3644731	total: 516ms	remaining: 21.6s
7:	learn: 1.3313474	total: 588ms	remaining: 21.5s
8:	learn: 1.2995905	total: 642ms	remaining: 20.8s
9:	learn: 1.2704823	total: 695ms	remaining: 20.1s
10:	learn: 1.2428384	total: 761ms	remaining: 20s
11:	learn: 1.2179029	total: 820ms	remaining: 19.7s
12:	learn: 1.1910441	total: 886ms	remaining: 19.6s
13:	learn: 1.1679649	total: 941ms	remaining: 19.2s
14:	learn: 1.1468280	total: 995ms	remaining: 18.9s
15:	learn: 1.1253543	total: 1.05s	remaining: 18.7s
16:	learn: 1.1057032	total: 1.12s	remaining: 18.6s
17:	learn: 1.0884668	total: 1.17s	remaining: 18.3s
18:	learn: 1.0719831	total: 1.22s	remaining: 18s
19:	learn: 1.0559773	total: 1.29s	remaining:

/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to mat

...

0:	learn: 1.2657897	total: 24ms	remaining: 7.18s
1:	learn: 1.0944216	total: 44.7ms	remaining: 6.66s
2:	learn: 1.0145659	total: 66.2ms	remaining: 6.56s
3:	learn: 0.9646178	total: 90.1ms	remaining: 6.67s
4:	learn: 0.9322892	total: 110ms	remaining: 6.49s
5:	learn: 0.9107357	total: 128ms	remaining: 6.27s
6:	learn: 0.8929944	total: 144ms	remaining: 6.03s
7:	learn: 0.8758478	total: 161ms	remaining: 5.87s
8:	learn: 0.8607470	total: 183ms	remaining: 5.91s
9:	learn: 0.8480815	total: 203ms	remaining: 5.9s
10:	learn: 0.8376047	total: 221ms	remaining: 5.8s
11:	learn: 0.8276697	total: 248ms	remaining: 5.96s
12:	learn: 0.8194627	total: 298ms	remaining: 6.58s
13:	learn: 0.8092001	total: 333ms	remaining: 6.81s
14:	learn: 0.8023636	total: 350ms	remaining: 6.66s
15:	learn: 0.7973689	total: 368ms	remaining: 6.53s
16:	learn: 0.7925128	total: 393ms	remaining: 6.55s
17:	learn: 0.7878264	total: 423ms	remaining: 6.62s
18:	learn: 0.7832914	total: 441ms	remaining: 6.52s
19:	learn: 0.7794587	total: 463ms	remaini

/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to mat

0:	learn: 1.2592702	total: 22.1ms	remaining: 6.6s
1:	learn: 1.0810425	total: 40.8ms	remaining: 6.08s
2:	learn: 0.9958325	total: 58.2ms	remaining: 5.76s
3:	learn: 0.9504889	total: 80.7ms	remaining: 5.97s
4:	learn: 0.9174390	total: 99.4ms	remaining: 5.86s
5:	learn: 0.8973356	total: 118ms	remaining: 5.8s
6:	learn: 0.8791750	total: 137ms	remaining: 5.75s
7:	learn: 0.8634577	total: 156ms	remaining: 5.71s
8:	learn: 0.8476277	total: 179ms	remaining: 5.79s
9:	learn: 0.8369909	total: 197ms	remaining: 5.72s
10:	learn: 0.8274529	total: 214ms	remaining: 5.61s
11:	learn: 0.8134890	total: 235ms	remaining: 5.65s
12:	learn: 0.8043734	total: 293ms	remaining: 6.47s
13:	learn: 0.7959457	total: 332ms	remaining: 6.79s
14:	learn: 0.7897380	total: 350ms	remaining: 6.65s
15:	learn: 0.7821860	total: 378ms	remaining: 6.71s
16:	learn: 0.7768674	total: 404ms	remaining: 6.72s
17:	learn: 0.7688752	total: 424ms	remaining: 6.64s
18:	learn: 0.7638289	total: 445ms	remaining: 6.57s
19:	learn: 0.7595300	total: 492ms	rema

/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)
/Users/charlesproye/opt/miniconda3/envs/boost/lib/python3.8/site-packages/sklearn/model_selection/_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan 0.812033

0:	learn: 1.5661333	total: 187ms	remaining: 55.8s
1:	learn: 1.4707010	total: 269ms	remaining: 40s
2:	learn: 1.3878090	total: 429ms	remaining: 42.4s
3:	learn: 1.3157268	total: 875ms	remaining: 1m 4s
4:	learn: 1.2525660	total: 937ms	remaining: 55.3s
5:	learn: 1.1979442	total: 1s	remaining: 49.3s
6:	learn: 1.1481995	total: 1.07s	remaining: 44.9s
7:	learn: 1.1048652	total: 1.17s	remaining: 42.9s
8:	learn: 1.0685387	total: 1.24s	remaining: 40s
9:	learn: 1.0361351	total: 1.32s	remaining: 38.2s
10:	learn: 1.0081768	total: 1.39s	remaining: 36.6s
11:	learn: 0.9834248	total: 1.49s	remaining: 35.7s
12:	learn: 0.9588818	total: 1.55s	remaining: 34.2s
13:	learn: 0.9362781	total: 1.67s	remaining: 34s
14:	learn: 0.9187983	total: 1.73s	remaining: 33s
15:	learn: 0.9023636	total: 1.8s	remaining: 32s
16:	learn: 0.8875679	total: 1.87s	remaining: 31.1s
17:	learn: 0.8750759	total: 1.97s	remaining: 30.8s
18:	learn: 0.8637519	total: 2.03s	remaining: 30.1s
19:	learn: 0.8536491	total: 2.13s	remaining: 29.8s
20:	